# T matrix for electromagnetic scattering

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [2]:
d0 = [0,0,-1] # wave's travel direction
p0 = [1,0,0] # polarization 
k = 4 # wavenumber
deg = 20

In [3]:
def coeff_inc(p,q):
    alpha_q = (0.5*(-1j)**q)*(2*q+1)/(q*(q+1))
    c_q_neg1 = np.sqrt(((2*q+1)/(4*np.pi))*(math.factorial(q-1)/math.factorial(q+1)))
    c_q_pos1 = -c_q_neg1
    if p == -1:
        return (-1j)*alpha_q / c_q_neg1,  alpha_q / c_q_neg1
    elif p == 1:
        return (1j)*alpha_q / c_q_pos1,  alpha_q / c_q_pos1
    else:
        return 0, 0

In [4]:
def sph_b(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)  

def sph_b_dr(q,x):
    r = np.linalg.norm(x)
    return k*(sph_b(q-1,x) - ((q+1)/(k*r))*sph_b(q,x))

In [5]:
def sph(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)
    
def sph_dazi(p, q, x):
    return 1j * p * sph(p, q, x)

In [6]:
def sph_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = sph(p, q, x)
    y3 = sph(-p, q, x)
    
    if abs(p) == q and p >= 0:
        return (p / np.tan(polar)) * y1 
    elif abs(p) != q and p >= 0:
        y2 = sph(p + 1, q, x)
        return (p / np.tan(polar)) * y1 - np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return ((-1)**p) * (-p / np.tan(polar)) * y3 * np.exp(1j*2*p*azimuth)
    elif abs(p) != q and p < 0:
        y4 = sph(-p + 1, q, x)
        return  ((-1)**p) * ((-p / np.tan(polar)) * y3 - np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(-1j * azimuth) * y4) * np.exp(1j*2*p*azimuth)

In [7]:
def sph_wf_M(p,q,x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    grad_sph = [sph_dpolar(p,q,x)*np.cos(polar)*np.cos(azimuth) - sph_dazi(p,q,x)*np.sin(azimuth)/np.sin(polar), 
                sph_dpolar(p,q,x)*np.cos(polar)*np.sin(azimuth) + sph_dazi(p,q,x)*np.cos(azimuth)/np.sin(polar),
               -sph_dpolar(p,q,x)*np.sin(polar)]
    hat_x = [np.sin(polar)*np.cos(azimuth), np.sin(polar)*np.sin(azimuth), np.cos(polar)]
    
    return sph_b(q,x)*np.cross(grad_sph, hat_x)

In [8]:
def sph_wf_N(p,q,x):
    r = np.linalg.norm(x)
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/r)
    
    grad_sph = [sph_dpolar(p,q,x)*np.cos(polar)*np.cos(azimuth) - sph_dazi(p,q,x)*np.sin(azimuth)/np.sin(polar), 
                sph_dpolar(p,q,x)*np.cos(polar)*np.sin(azimuth) + sph_dazi(p,q,x)*np.cos(azimuth)/np.sin(polar),
               -sph_dpolar(p,q,x)*np.sin(polar)]
    hat_x = [np.sin(polar)*np.cos(azimuth), np.sin(polar)*np.sin(azimuth), np.cos(polar)]
    
    
    part_1 = [(q*(q+1)/r)*sph_b(q,x)*sph(p,q,x)*hat_x[0], 
             (q*(q+1)/r)*sph_b(q,x)*sph(p,q,x)*hat_x[1], 
             (q*(q+1)/r)*sph_b(q,x)*sph(p,q,x)*hat_x[2]]
    
    part_2 = [(1/r)*(sph_b(q,x)+r*sph_b_dr(q,x))*grad_sph[0], 
                (1/r)*(sph_b(q,x)+r*sph_b_dr(q,x))*grad_sph[1],
               (1/r)*(sph_b(q,x)+r*sph_b_dr(q,x))*grad_sph[2]]
    
    return [(1/(1j*k))*(part_1[0]+part_2[0]), (1/(1j*k))*(part_1[1]+part_2[1]), (1/(1j*k))*(part_1[2]+part_2[2])]

In [9]:
coeff_list_temp = np.zeros(((deg+1)**2-1,2),dtype = complex)
i = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        coeff_list_temp[i,:] = coeff_inc(p,q)
        i += 1
coeff_list = np.hstack((coeff_list_temp[:,0],coeff_list_temp[:,1]))

In [10]:
test_pt = [1,2,3]

fun_value_list_temp1 = np.zeros(((deg+1)**2-1,3),dtype = complex)
j1 = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        fun_value_list_temp1[j1,:] = sph_wf_M(p,q,test_pt)
        j1 += 1
        
fun_value_list_temp2 = np.zeros(((deg+1)**2-1,3),dtype = complex)
j2 = 0
for q in range(1,deg+1):
    for p in range(-q,q+1):
        fun_value_list_temp2[j2,:] = sph_wf_N(p,q,test_pt)
        j2 += 1
        
fun_value_list = np.vstack((fun_value_list_temp1,fun_value_list_temp2))

In [11]:
inc_appro = [0,0,0]
inc_appro[0] = np.dot(coeff_list,fun_value_list[:,0])
inc_appro[1] = np.dot(coeff_list,fun_value_list[:,1])
inc_appro[2] = np.dot(coeff_list,fun_value_list[:,2])

print(inc_appro)

[(0.8435099024832893+0.5413891619746731j), (-0.0027315409610968434-0.0010150126908399233j), (0.0006417386939819939+0.000525131930917051j)]


In [12]:
inc_exact =  [np.exp(1j*k*(-3)),0 ,0]
print(inc_exact)

[(0.8438539587324921+0.5365729180004349j), 0, 0]
